# Corona Health - Physiological Health of adults (`Heart`) Study

## Purpose of this Notebook
- [ ] Clean the dataset and save cleaned version
- [x] Get an statistical overview
    - [x] How many users?
    - [x] How many assessments (= filled out questionnaires)?
    - [x] Date range of the dataset?
    - [x] User-assessment distribution
- [ ] Potential target for classification?
- [ ] Potential features for classification?
    
    

## Preparation
Import modules and load data for later use

In [2]:
# imports
import pandas as pd
from datetime import date, datetime

In [3]:
# to import own modules, a workaround - assuming the curent working directory is the folder (your/local/path/UsAs/src/d01_analysis
import sys
sys.path.insert(0, "../..")

from src.d00_utils import cc_helpers, helpers

In [4]:
# read in dataframe
# assuming current working directory is the folder (your/local/path/UsAs/src/d01_analysis)
df = pd.read_csv('../../data/d01_raw/ch/22-10-05_rki_heart_followup.csv')
df_baseline = pd.read_csv('../../data/d01_raw/ch/22-10-05_rki_heart_baseline.csv')

## Statistical overview
Here we calculate statistics for both the dataset overall and the baseline questionaires

In [5]:
# create result dict to save to disk
result = dict()

### Number of users

In [6]:
# Number of unique users
print('Number of user:\t', df.user_id.nunique())
result['n_users'] = df.user_id.nunique()

Number of user:	 953


### Number of assessments

In [7]:
# number of unique answers
print('Number of assessments:\t', df.answer_id.nunique())
result['n_assessments'] = df.answer_id.nunique()

Number of assessments:	 5661


### Date Range of assessments

In [8]:
# date range
form = '%Y-%m-%d %H:%M:%S'
date_start = df.created_at.min()
result['First assessment from'] = date_start
date_start = datetime.strptime(date_start, form)
date_end = df.created_at.max()
result['Latest assessment from'] = date_end
date_end = datetime.strptime(date_end, form)



print('Start:\t', date_start)
print('End:\t', date_end)

delta = date_end.date()-date_start.date()

print('\nDate Range in')
print('Years:\t', delta.days/365)
print('Months:\t', delta.days/12)
print('Days:\t', delta.days)


result['Time range in days'] = delta.days

Start:	 2020-07-28 13:55:53
End:	 2022-09-29 18:48:46

Date Range in
Years:	 2.1726027397260275
Months:	 66.08333333333333
Days:	 793


### Schedule pattern

In [9]:
helpers.find_schedule_pattern(df)

{'Median hours between two assessments': 336.72055555555556,
 'Median days between two assessments': 14.03002314814815,
 'std_hours': 629.2315549885324,
 'std_days': 26.217981457855515}

### Distribution of filled out questionnaires

In [10]:
bins = [0, 1, 2, 3, 5, 10, 100, 1000]
ser = pd.cut(df.user_id.value_counts(), bins = bins).value_counts().sort_index()
ser
result.update(dict(ser))

### Average period length between two filled out questionnaires

In [11]:
# res = result
res = helpers.find_schedule_pattern(df, form='%Y-%m-%d %H:%M:%S', date_col_name='created_at')
result.update(res)

### Age Distribution

The following table shows the number of users for each age.

In [12]:
df_baseline.groupby('alter')['user_id'].count()

alter
18.0     20
19.0     11
20.0      9
21.0     16
22.0     16
         ..
79.0      1
80.0      1
82.0      1
107.0     1
120.0     1
Name: user_id, Length: 63, dtype: int64

Now we compute the mean age and the standard deviation

In [13]:
avg_age = df_baseline['alter'].mean()
std_age = df_baseline['alter'].std()

result['avg_age'] = avg_age
result['std_age'] = std_age

print("Durschnittliches Alter: %4.2f Jahre" % avg_age)
print("Standardabweichung Alter: %4.2f Jahre" % std_age)

Durschnittliches Alter: 41.80 Jahre
Standardabweichung Alter: 15.16 Jahre


### Sex Distribution

We calculate the number of users that self identified their sex as male (0), female (1) or other (2)

In [14]:
df_sex = pd.DataFrame(df_baseline.groupby('geschlecht')['user_id'].count())
df_sex['label'] = ('male', 'female', 'other')

result['n_users_male'] = df_sex['user_id'][0.0]
result['n_users_female'] = df_sex['user_id'][1.0]
result['n_users_other'] = df_sex['user_id'][2.0]

print(df_sex)
print("\n{} users without submitted sex".format(df_baseline['user_id'].count() - (result['n_users_male'] + result['n_users_female'] + result['n_users_other'])))

            user_id   label
geschlecht                 
0.0             415    male
1.0             241  female
2.0               3   other

1121 users without submitted sex


### Country Statistics

We calculate how many users participated by country

In [15]:
country_series = df_baseline.groupby('country')['user_id'].count()
country_series

country
AND      1
ATG      1
AUT      3
DEU    650
FRA      1
GBR      1
MEX      1
NLD      1
Name: user_id, dtype: int64

We also calculate the percentage of german-based users in the dataset

In [16]:
result['n_users_german'] = country_series['DEU']
result['n_users_all'] = country_series.sum()
print('{:.2f}% german-based users in dataset'.format(country_series['DEU'] / country_series.sum() * 100))

98.63% german-based users in dataset


### Statistical overview

In [17]:
result

{'n_users': 953,
 'n_assessments': 5661,
 'First assessment from': '2020-07-28 13:55:53',
 'Latest assessment from': '2022-09-29 18:48:46',
 'Time range in days': 793,
 Interval(0, 1, closed='right'): 311,
 Interval(1, 2, closed='right'): 138,
 Interval(2, 3, closed='right'): 105,
 Interval(3, 5, closed='right'): 119,
 Interval(5, 10, closed='right'): 128,
 Interval(10, 100, closed='right'): 152,
 Interval(100, 1000, closed='right'): 0,
 'Median hours between two assessments': 336.72055555555556,
 'Median days between two assessments': 14.03002314814815,
 'std_hours': 629.2315549885324,
 'std_days': 26.217981457855515,
 'avg_age': 41.79514415781487,
 'std_age': 15.156495917006229,
 'n_users_male': 415,
 'n_users_female': 241,
 'n_users_other': 3,
 'n_users_german': 650,
 'n_users_all': 659}

**Questionnaire of most frequent user has been filled out ~ each 14 days, i.e. one period ~ 14 days**

In [18]:
# read in codebook and reduce to columns and rows of interest
cb = pd.read_excel('../../data/d00_helpers/codebook/ch/rki_heart.xlsx', sheet_name='FollowUp', header=4)
cb = cb[cb.elementtype=='question']
cb = cb.iloc[:, :30]

FileNotFoundError: [Errno 2] No such file or directory: '../../data/d00_helpers/codebook/ch/rki_heart.xlsx'